In [4]:
!pip uninstall opengradient

Found existing installation: opengradient 0.1.0
Can't uninstall 'opengradient'. No files were found to uninstall.


In [15]:
!python setup.py sdist bdist_wheel
!pip install dist/opengradient-0.1.0.tar.gz

running sdist
running egg_info
writing opengradient.egg-info/PKG-INFO
writing dependency_links to opengradient.egg-info/dependency_links.txt
writing requirements to opengradient.egg-info/requires.txt
writing top-level names to opengradient.egg-info/top_level.txt
reading manifest file 'opengradient.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'opengradient.egg-info/SOURCES.txt'
running check
creating opengradient-0.1.0
creating opengradient-0.1.0/opengradient.egg-info
creating opengradient-0.1.0/src
creating opengradient-0.1.0/tests
copying files to opengradient-0.1.0...
copying LICENSE -> opengradient-0.1.0
copying README.md -> opengradient-0.1.0
copying setup.py -> opengradient-0.1.0
copying opengradient.egg-info/PKG-INFO -> opengradient-0.1.0/opengradient.egg-info
copying opengradient.egg-info/SOURCES.txt -> opengradient-0.1.0/opengradient.egg-info
copying opengradient.egg-info/dependency_links.txt -> opengradient-0.1.0/opengradient.egg-info
copying openg

In [4]:
!pip install -r requirements.txt

In [5]:
from opengradient import Client
from opengradient.types import ModelInput, InferenceMode
import json

client = Client(wallet_address="0xeB155EcC59fcc2eBdeF6dEa1711597A5dacEfB30", private_key="cd09980ef6e280afc3900d2d6801f9e9c5d858a5deaeeab74a65643f5ff1a4c1")
client.upload("network.onnx")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 18.222.64.142:5000
DEBUG:urllib3.connectionpool:http://18.222.64.142:5000 "POST /upload HTTP/11" 400 12


UploadError: Upload failed: 400 Client Error: BAD REQUEST for url: http://18.222.64.142:5000/upload (Status code: None)

In [1]:
from opengradient import Client
from opengradient.types import ModelInput, InferenceMode, Number, NumberTensor, ModelOutput
import json
import web3

print("Step 1: Importing modules")
print(f"ModelInput attributes: {ModelInput.__dict__}")

print("\nStep 2: Loading input data from JSON file")
with open('input.json', 'r') as file:
    input_data = json.load(file)
print(f"Loaded input data: {input_data}")

print("\nStep 3: Initializing client")
client = Client(wallet_address="0xeB155EcC59fcc2eBdeF6dEa1711597A5dacEfB30", private_key="cd09980ef6e280afc3900d2d6801f9e9c5d858a5deaeeab74a65643f5ff1a4c1")
print("Client initialized")
print(f"Client attributes: {client.__dict__}")

print("\nStep 4: Preparing model input")
import numpy as np
try:
    input_values = input_data['input_data'][0]
    reshaped_input = np.array(input_values).reshape(1, 3, 2, 2)  # Reshape to [batch_size, channels, height, width]
    number_tensor = NumberTensor(
        name="input",
        values=[Number(value=int(value * 1e6), decimals=6) for value in reshaped_input.flatten()]
    )
    model_input = ModelInput(
        numbers=[
            NumberTensor(
                name="input",
                values=[
                    Number(value=1111111111111111, decimals=17),
                    Number(value=4486280083656311, decimals=16)
                ]
            )
        ],
        strings=[]  # This should be a list of StringTensor objects if needed
    )

    print(f"ModelInput type: {type(model_input)}")
    print(f"NumberTensor type: {type(model_input.numbers[0])}")
    print(f"Number type: {type(model_input.numbers[0].values[0])}")
    print(f"Number value type: {type(model_input.numbers[0].values[0].value)}")
    print(f"ModelInput created successfully: {model_input}")
except Exception as e:
    print(f"Error creating ModelInput: {str(e)}")
    raise

print("\nStep 5: Setting model ID and inference mode")
model_id = "QmbbzDwqSxZSgkz1EbsNHp2mb67rYeUYHYWJ4wECE24S7A"
inference_mode = InferenceMode.VANILLA
print(f"Model ID: {model_id}")
print(f"Inference mode: {inference_mode}")

print("\nStep 6: Checking Web3 initialization")
if hasattr(client, '_initialize_web3'):
    print("Calling _initialize_web3 method")
    try:
        client._initialize_web3()
    except AttributeError:
        print("AttributeError occurred during _initialize_web3. Attempting to modify the method.")
        def _initialize_web3(self):
            from web3 import Web3
            self.w3 = Web3(Web3.HTTPProvider(self.rpc_url))
            print(f"Web3 connection established: {self.w3.is_connected()}")
        client._initialize_web3 = _initialize_web3.__get__(client)
        client._initialize_web3()
else:
    print("_initialize_web3 method not found")

print(f"Client w3 attribute after initialization: {getattr(client, 'w3', 'Not found')}")
print("\nStep 7: Calling infer method")
try:
    if client.w3 is None:
        print("Web3 connection is None. Attempting to reinitialize...")
        from web3 import Web3
        client.w3 = Web3(Web3.HTTPProvider(client.rpc_url))
        print(f"Web3 connection reinitialized: {client.w3.is_connected()}")
    
    print(f"Ethereum node version: {client.w3.client_version}")
    
    print("Calling infer method with the following parameters:")
    print(f"Model ID: {model_id}")
    print(f"Inference Mode: {inference_mode}")
    print(f"Modified Model Input: {model_input}")
    
    result, tx_hash, inference_id = client.infer(model_id, inference_mode, model_input)
    
    print("Infer method completed. Raw results:")
    print(f"Result: {result}")
    print(f"Transaction hash: {tx_hash}")
    print(f"Inference ID: {inference_id}")
    
    if isinstance(result, ModelOutput):
        print("ModelOutput details:")
        print(f"Numbers: {result.numbers}")
        print(f"Strings: {result.strings}")
        print(f"Is simulation result: {result.is_simulation_result}")
    else:
        print(f"Unexpected result type: {type(result)}")

except web3.exceptions.Web3RPCError as e:
    print(f"Web3 RPC Error occurred: {str(e)}")
    print(f"Error code: {e.code}")
    print(f"Error message: {e.message}")
except Exception as e:
    print(f"An error occurred during inference: {str(e)}")
    print(f"Error type: {type(e)}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())

DEBUG:web3.manager.RequestManager:Making request. Method: web3_clientVersion
DEBUG:web3.providers.HTTPProvider:Making request HTTP. URI: http://18.218.115.248:8545, Method: web3_clientVersion
DEBUG:web3._utils.http_session_manager.HTTPSessionManager:Session cached: http://18.218.115.248:8545, <requests.sessions.Session object at 0x113a5b6e0>
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 18.218.115.248:8545
DEBUG:urllib3.connectionpool:http://18.218.115.248:8545 "POST / HTTP/11" 200 43
DEBUG:web3.providers.HTTPProvider:Getting response HTTP. URI: http://18.218.115.248:8545, Method: web3_clientVersion, Response: {'jsonrpc': '2.0', 'id': 0, 'result': '0.38.5'}
DEBUG:root:Entering infer method
DEBUG:web3.providers.HTTPProvider:Making request HTTP. URI: http://18.218.115.248:8545, Method: web3_clientVersion
DEBUG:urllib3.connectionpool:http://18.218.115.248:8545 "POST / HTTP/11" 200 43
DEBUG:web3.providers.HTTPProvider:Getting response HTTP. URI: http://18.218.115.248:8545,

Private Key: c04da81ab8e2fcaa6050ff361bf409d95a3d37149c2b748a16e64265d10f1200
Ethereum Address: 0x84Da62f1eaE62982280C957d6bcAADb009901688
Step 1: Importing modules
ModelInput attributes: {'__module__': 'opengradient.types', '__annotations__': {'numbers': typing.List[opengradient.types.NumberTensor], 'strings': typing.List[opengradient.types.StringTensor]}, '__dict__': <attribute '__dict__' of 'ModelInput' objects>, '__weakref__': <attribute '__weakref__' of 'ModelInput' objects>, '__doc__': 'ModelInput(numbers: List[opengradient.types.NumberTensor], strings: List[opengradient.types.StringTensor])', '__dataclass_params__': _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False,match_args=True,kw_only=False,slots=False,weakref_slot=False), '__dataclass_fields__': {'numbers': Field(name='numbers',type=typing.List[opengradient.types.NumberTensor],default=<dataclasses._MISSING_TYPE object at 0x103b30fb0>,default_factory=<dataclasses._MISSING_TYPE object at 

DEBUG:urllib3.connectionpool:http://18.218.115.248:8545 "POST / HTTP/11" 200 1493
DEBUG:web3.providers.HTTPProvider:Getting response HTTP. URI: http://18.218.115.248:8545, Method: eth_getBlockByNumber, Response: {'jsonrpc': '2.0', 'id': 3, 'result': {'difficulty': '0x0', 'extraData': '0x', 'gasLimit': '0xffffffffffffffff', 'gasUsed': '0x0', 'hash': '0xb4d201e5b8389d15e5f0aec15c532726a9a109bc6f957625742aa3da8c17f63a', 'logsBloom': '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', 'miner': '0x000000000000000000000000000000000000

An error occurred during inference: Inference failed: {'code': -32000, 'message': 'tx failed with code: 19: raw_log: : invalid request'} (Status code: None)
Error type: <class 'src.exceptions.api_exceptions.InferenceError'>
Traceback:
Traceback (most recent call last):
  File "/Users/oliver/Desktop/code/opengradient/sdk/src/client.py", line 134, in infer
    tx_hash = self.w3.eth.send_raw_transaction(signed_tx.raw_transaction)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/vanna-storage/lib/python3.12/site-packages/web3/eth/eth.py", line 370, in send_raw_transaction
    return self._send_raw_transaction(transaction)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/vanna-storage/lib/python3.12/site-packages/web3/module.py", line 111, in caller
    result = w3.manager.request_blocking(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/vanna-storage/lib/python3.12/site-packages/web3